In [1]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from string import digits
import numpy as np 
import pandas as pd
from PIL import Image
import pickle

In [2]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## Import Data
Get dataset and split into individual sentences

Create word corpus (vocabulary list from dataset)

Calculate total number of unique words, +1 for the padding "words"

In [3]:
with open('data/captions.txt') as f:
    captions = [lines.strip().split(',') for lines in f.readlines()]

captions = captions[1:]
f.close()

In [4]:
captions = pd.DataFrame(captions)
captions.head(1)

,0,1,2,3,4,5,6
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...,None,None,None,None,None


In [5]:
captions.describe()

,0,1,2,3,4,5,6
count,40455,40455,2447,715,64,5,1
unique,8091,39929,2288,705,64,5,1
top,1285067106_2adc307240.jpg,"""Two dogs","""","and a backpack .""","are in a mall .""","cover their mouths .""",are waving to the crowd along the parade rout...
freq,5,37,19,3,1,1,1


## Data Cleansing For The Captions

In [6]:
# idx_lines_split = []
# for i in [2,3,4,5,6]:
#     idx_lines_split += list(captions[captions[i].notnull()].index)

# idx_lines_split = list(set(idx_lines_split))

# remove_digits = str.maketrans('', '', digits)
# for index in idx_lines_split:
#     # merging the sentences that were split into multiple columns into a string
#     merged_caption = " ".join(captions.iloc[index,[1,2,3,4,5,6]].fillna("").values).strip()
#     captions.iloc[index,1] = merged_caption

# cleaned_captions = []
# for i in range(len(captions)):
#     # removing numbers from the string
#     cleaned_caption = captions.iloc[i,1].translate(remove_digits)
#     cleaned_caption = "starttoken " + cleaned_caption + " endtoken"
#     cleaned_captions.append(cleaned_caption)

In [7]:
# with open("data/cleaned_captions.pickle", 'wb') as file_pi:
#     pickle.dump(cleaned_captions, file_pi)

with open("data/cleaned_captions.pickle", "rb") as f:
    cleaned_captions = pickle.load(f)

In [8]:
# encoder_model = tf.keras.applications.Xception(
#     weights='imagenet',  # Load weights pre-trained on ImageNet.
#     input_shape=(299, 299, 3),
#     pooling='avg',
#     include_top=False)  # Do not include the ImageNet classifier at the top.

In [9]:
# # every 5 rows, extract the image linked to those 5 captions
# img_features = []
# for i in range(0,len(captions),5):
#     # reading the file
#     filename = captions.iloc[i,0]
#     img = Image.open("data/Images/" + filename)
#     # resizing images to 299 by 299 as its the default model input size
#     img = img.resize((299,299))
#     img = np.array(img)
#     # preprocess image pixel values to -1 to 1
#     img = np.expand_dims(img, axis=0)
#     img = tf.keras.applications.xception.preprocess_input(img)
#     # extract feature vectors using the encoder CNN model
#     features = encoder_model.predict(img)
#     img_features.append(features)

# with open("data/image_features.pickle", 'wb') as file_pi:
#     pickle.dump(img_features, file_pi)

In [10]:
with open("data/image_features.pickle", "rb") as x:
    img_features = pickle.load(x)

In [11]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(cleaned_captions)
total_words = len(tokenizer.word_index) + 1 # +1 because of padding

In [12]:
input_sequences = []
caption_lengths = []
for line in cleaned_captions: # for each sentence in dataset
	# line = "starttoken " + cleaned_captions[i] + " endtoken"
	token_list = tokenizer.texts_to_sequences([line])[0] # tokenize each sentence
	num_words = len(token_list)
	for j in range(1, num_words): # create n-grams for each sentence so that
		n_gram_sequence = token_list[:j+1]
		input_sequences.append(n_gram_sequence)

	caption_lengths.append(num_words)

# pad sequences so that inputs will have same length
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label (last word in sentence)
sequence, labels = input_sequences[:,:-1],input_sequences[:,-1]

labels = tf.keras.utils.to_categorical(labels, num_classes=total_words) #convert them to categorical and one hot encode them so that it can predict which word from the vocab it is

In [17]:
max_sequence_len

38

In [44]:
training_img_features = []
for i in range(0,len(img_features)):
    # find out how many times each image must appear in the dataset to match the number of n-grams for each image
    corresponding_caption_idx = i * 5
    num_repeating_imgs = sum(caption_lengths[corresponding_caption_idx:corresponding_caption_idx+5]) - 5 # -5 because each caption has 1 less n-gram than words and there are 5 captions
    for j in range(num_repeating_imgs):
        training_img_features.append(img_features[i][0])

In [45]:
print(len(training_img_features))
print(len(sequence))
print(len(labels))

477906
477906
477906


In [ ]:
class ImageCaptioningModel:
    def __init__(self, total_words, input_length):
        self.total_words = total_words
        self.input_length = input_length
    def build_encoder(self):
        text_input = keras.Input(shape=(self.input_length,))

        x = Embedding(self.total_words, 100, input_length=self.input_length)(text_input)
        x = Bidirectional(LSTM(150))(x)

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1)) #embedding includes start token
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax')) # model shjld not predict start token so -1
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(sequence, labels, epochs=100, verbose=1)
#print model.summary()
print(model)

In [ ]:
    def get_discriminator(self):
        image_input = Input(shape=self.image_size)

        x = self.conv_block(image_input, 128, 4, 2, 'same', use_batch_norm=True, use_bias=False, activation=LeakyReLU(alpha=leaky_relu_slope))
        x = self.conv_block(x, 128, 4, 2, 'same', use_batch_norm=True, use_bias=False, activation=LeakyReLU(alpha=leaky_relu_slope))
        x = self.conv_block(x, 128, 4, 2, 'same', use_batch_norm=True, use_bias=False, activation=LeakyReLU(alpha=leaky_relu_slope))

        x = Flatten()(x)
        x = Dropout(0.4)(x)
        output_score = Dense(1)(x)

        return keras.Model(image_input, output_score, name="discriminator")

In [ ]:
seed_text = "I've got a bad feeling about this"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = np.argmax(model.predict(token_list), axis=-1)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

stuff